<a href="https://colab.research.google.com/github/vimesh630/Revenue_Forecasting/blob/main/Prophet_for_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Mount Google Drive & Import Required Libraries

In [61]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from prophet import Prophet
from tqdm import tqdm
from datetime import datetime
from dateutil.relativedelta import relativedelta

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#2. Load Dataset

In [62]:
CSV_PATH = "/content/drive/MyDrive/VERGER/Revenue_Forecasting/forecasting_data.csv"
TARGET_YEAR  = 2027
TARGET_MONTH = 3

# Prophet settings
MIN_POINTS_FOR_PROPHET = 2        # Minimum months of history to fit Prophet well
FREQ = "MS"
ROLLING_PRICE_WINDOW = 2
HIST_ACTIVE_MONTHS = 6

#3. Data Preprocessing

In [63]:
df = pd.read_csv(CSV_PATH)

# Ensure Month_No exists
if "Month_No" in df.columns:
    month_num = df["Month_No"].astype(pd.Int64Dtype()).fillna(1)
else:
    month_map = {m: i for i, m in enumerate(
        ["January","February","March","April","May","June","July","August","September","October","November","December"], start=1)}
    month_num = df["Month"].map(month_map).astype(pd.Int64Dtype()).fillna(1)

df["Year"] = df["Year"].astype(pd.Int64Dtype()).fillna(1)
df["date"] = pd.to_datetime(dict(year=df["Year"], month=month_num, day=1), errors='coerce')
df = df.dropna(subset=["date"])
df = df[["date","Year","Month","Account","Product","Type","Quantity","Unit_Price","Revenue"]].copy()

# Convert numeric columns
df["Quantity"] = pd.to_numeric(df["Quantity"], errors='coerce').clip(lower=0).fillna(0)
df["Unit_Price"] = pd.to_numeric(df["Unit_Price"], errors='coerce').clip(lower=0).fillna(0)
df["Revenue"] = pd.to_numeric(df["Revenue"], errors='coerce').clip(lower=0).fillna(0)

#4. Aggregate to monthly per series(handle multiple orders in same month)

In [64]:
agg = (df.groupby(["Account","Product","Type","date"], as_index=False)
       .agg(Quantity=("Quantity","sum"),
            Revenue=("Revenue","sum"),
            Unit_Price=("Unit_Price",
                        lambda x: (x * df.loc[x.index, "Quantity"]).sum() / df.loc[x.index, "Quantity"].sum()
                        if df.loc[x.index, "Quantity"].sum() > 0 else x.mean())
           ))
agg["Unit_Price"] = agg["Unit_Price"].fillna(0)
agg = agg[["Account","Product","Type","date","Quantity","Unit_Price","Revenue"]]

#5. Build complete monthly timelines (fill missing months with 0 qty)

In [65]:
last_hist_date = agg["date"].max()
active_cutoff = last_hist_date - relativedelta(months=HIST_ACTIVE_MONTHS)
all_series_keys = (agg.groupby(["Account","Product","Type"])
                   .filter(lambda x: (x["Quantity"][x["date"]>active_cutoff].sum() > 0) and len(x) >= MIN_POINTS_FOR_PROPHET)
                   [["Account","Product","Type"]]
                   .drop_duplicates()
                   .reset_index(drop=True))

global_min_date = agg["date"].min()
global_max_date = agg["date"].max()
target_date = pd.Timestamp(year=TARGET_YEAR, month=TARGET_MONTH, day=1)
if target_date <= global_max_date:
    raise ValueError(f"Target {target_date.strftime('%Y-%m')} is not after your last data month {global_max_date.strftime('%Y-%m')}.")

def month_range(start, end, freq="MS"):
    return pd.date_range(start=start, end=end, freq=freq)

calendar = pd.DataFrame({"date": month_range(global_min_date, target_date, FREQ)})

# Product-level median price fallback
product_price_stats = (agg.groupby(["Product"])
                       .agg(product_median_price=("Unit_Price","median"))
                       .reset_index())

#6. Helper Functions

In [66]:
def prepare_series(full_df, account, product, typ):
    sub = full_df[(full_df["Account"]==account) & (full_df["Product"]==product) & (full_df["Type"]==typ)].copy()
    s = calendar.merge(sub, on="date", how="left")
    s["Account"] = account
    s["Product"] = product
    s["Type"] = typ
    for col in ["Quantity","Revenue","Unit_Price"]:
        s[col] = s[col].fillna(0 if col != "Unit_Price" else np.nan)
    s["Unit_Price"] = s["Unit_Price"].ffill().bfill()
    return s

def seasonal_naive_forecast(history_df, forecast_months):
    hist = history_df.set_index("date")["Quantity"].copy()
    preds = []
    for i in range(1, forecast_months+1):
        future_month = history_df["date"].max() + relativedelta(months=i)
        same_month_last_year = future_month - relativedelta(years=1)
        if same_month_last_year in hist.index:
            preds.append(float(hist.loc[same_month_last_year]))
        else:
            last_nonzero = hist[hist > 0].tail(3)
            preds.append(float(last_nonzero.mean()) if len(last_nonzero) > 0 else 0.0)
    return np.array(preds, dtype=float)

def estimate_future_price(series_df, horizon_months, product_level_stats):
    s = series_df.copy()
    s["price_for_roll"] = s["Unit_Price"].where(s["Unit_Price"]>0)
    recent = s.dropna(subset=["price_for_roll"]).tail(ROLLING_PRICE_WINDOW)
    if not recent.empty:
        base_price = float(recent["price_for_roll"].median())
    else:
        prod = s["Product"].iloc[0]
        row = product_level_stats[product_level_stats["Product"]==prod]
        if not row.empty and not np.isnan(row["product_median_price"].iloc[0]):
            base_price = float(row["product_median_price"].iloc[0])
        else:
            last_known = s["Unit_Price"][s["Unit_Price"]>0].tail(1)
            base_price = float(last_known.iloc[0]) if not last_known.empty else 0.0
    return np.full(shape=(horizon_months,), fill_value=base_price, dtype=float)

def prophet_forecast_quantity(series_df, forecast_months):
    hist = series_df[series_df["date"] <= global_max_date].copy()
    df_p = pd.DataFrame({"ds": hist["date"], "y": hist["Quantity"].astype(float)})
    m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False,
                changepoint_prior_scale=0.5, seasonality_mode="additive")
    m.fit(df_p)
    fut = m.make_future_dataframe(periods=forecast_months, freq="MS")
    fcst = m.predict(fut)
    mask = fcst["ds"] > global_max_date
    yhat = fcst.loc[mask, "yhat"].to_numpy(dtype=float)
    return np.clip(yhat, 0, None)

def clip_forecast(series_df, forecast):
    hist_avg = series_df["Quantity"].mean()
    hist_max = series_df["Quantity"].max()
    # Clip forecast to realistic range: max 1.5x historical max or 3x historical mean
    return np.minimum(forecast, max(hist_max*1.5, hist_avg*3))

# Forecast horizon
delta = relativedelta(target_date, global_max_date)
months_to_forecast = delta.years * 12 + delta.months + 1

#7. Forecasting Loop

In [67]:
results = []
for idx, row in tqdm(all_series_keys.iterrows(), total=len(all_series_keys), desc="Forecasting"):
    acc, prod, typ = row["Account"], row["Product"], row["Type"]
    s = prepare_series(agg, acc, prod, typ)
    s_hist = s[s["date"] <= global_max_date]
    n_points = len(s_hist)

    # Choose model
    use_prophet = n_points >= MIN_POINTS_FOR_PROPHET and s_hist["Quantity"].sum() > 0
    if use_prophet:
        try:
            qty_preds = prophet_forecast_quantity(s, months_to_forecast)
            qty_preds = clip_forecast(s_hist, qty_preds)
        except:
            qty_preds = seasonal_naive_forecast(s_hist, months_to_forecast)
    else:
        qty_preds = seasonal_naive_forecast(s_hist, months_to_forecast)

    # Price prediction
    price_preds = estimate_future_price(s, months_to_forecast, product_price_stats)

    # Build forecast records
    future_months = [(global_max_date + relativedelta(months=i)).replace(day=1)
                     for i in range(1, months_to_forecast+1)]
    for d, q, p in zip(future_months, qty_preds, price_preds):
        results.append({
            "Year": d.year,
            "Month": f"{d.month:02d}",
            "Account": acc,
            "Product": prod,
            "Type": typ,
            "Forecast_Quantity": float(q),
            "Forecast_Revenue": float(q) * float(p)
        })


Forecasting:   0%|          | 0/35 [00:00<?, ?it/s]DEBUG:cmdstanpy:input tempfile: /tmp/tmpa3_0xx1n/528nu6yi.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpa3_0xx1n/wzu1ug6h.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=64557', 'data', 'file=/tmp/tmpa3_0xx1n/528nu6yi.json', 'init=/tmp/tmpa3_0xx1n/wzu1ug6h.json', 'output', 'file=/tmp/tmpa3_0xx1n/prophet_model79uuhmhj/prophet_model-20250825035530.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
03:55:30 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
03:55:31 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
Forecasting:   3%|▎         | 1/35 [00:00<00:29,  1.15it/s]DEBUG:cmdstanpy:input tempfile: /tmp/tmpa3_0xx1n/iseg1ylj.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpa3_0xx1n/oaxr006c.json
DEBUG

#8. Results & Save

In [68]:
out = pd.DataFrame(results)
out["Forecast_Quantity"] = out["Forecast_Quantity"].round(2)
out["Forecast_Revenue"]  = out["Forecast_Revenue"].round(2)

# ✅ Shuffle output so it’s not alphabetical
out = out.sort_values(by=["Year", "Month"]).reset_index(drop=True)

display(out)  # just to preview

# Save to Drive
OUT_PATH = f"/content/drive/MyDrive/VERGER/Revenue_Forecasting/customer_level_forecast_{target_date.strftime('%Y_%m')}.csv"
out.to_csv(OUT_PATH, index=False)
print(f"Saved: {OUT_PATH}")

,Year,Month,Account,Product,Type,Forecast_Quantity,Forecast_Revenue
0,2025,02,AGFR,Cinnamon Bark Sri Lanka EO,Conventional,8.71,2178.36
1,2025,02,BLFR,Cinnamon Leaf Sri Lanka EO,Conventional,716.88,13262.31
2,2025,02,CNGR,Cinnamon Bark EO Org.,Organic,285.55,58538.15
3,2025,02,CNGR,Cinnamon Bark Sri Lanka EO,Conventional,18.06,3130.66
4,2025,02,CNGR,Pepper Black Sri Lanka EO,Conventional,269.44,8056.16
...,...,...,...,...,...,...,...
940,2027,04,TKGL,Clove Bud Sri Lanka EO,Conventional,50.71,3164.21
941,2027,04,TKGL,Clove Bud Sri Lanka EO FFL,Conventional,81.65,4082.73
942,2027,04,TKGL,Ginger Fresh Sri Lanka EO Org,Organic,0.09,45.90
943,2027,04,VDSP,Cinnamon Leaf Sri Lanka EO,Conventional,786.39,15531.26


Saved: /content/drive/MyDrive/VERGER/Revenue_Forecasting/customer_level_forecast_2027_03.csv
